<a href="https://colab.research.google.com/github/HJoon-K/Find_Dust/blob/main/fine_dust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (10.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [1]:
pip install pandas yfinance plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import json

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium

In [3]:
# 경고 메세지 감춤
import warnings
warnings.filterwarnings('ignore')

In [4]:
plt.rcParams['font.family'] = 'NanumGothic'

## 데이터 전처리

In [ ]:
# 데이터 합치기
files = ['dust2020.csv','dust2019.csv','dust2018.csv','dust2017.csv','dust2016.csv']
dust = pd.DataFrame()
for idx, file in enumerate(files):
    dust1 = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/data/{file}')
    if idx == 0:
        dust = dust.append(dust1)
    else:
        dust = dust.append(dust1, ignore_index=True)

dust.tail()

In [6]:
dust.to_csv('/content/drive/MyDrive/Colab Notebooks/data/dust.csv',index=False)

In [7]:
dust2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/dust.csv')

### 구별 데이터 처리

In [8]:
# 구별 미세먼지 : 나쁨 이상 횟수
dust3 = dust2[dust2['PM10'] > 80]
dust_gu = pd.DataFrame(dust3['gu'].value_counts())
dust_gu = dust_gu.reset_index()
dust_gu.rename(columns=({'index':'자치구', 'gu':'미세먼지나쁨일수'}), inplace=True)
dust_gu.to_csv('/content/drive/MyDrive/Colab Notebooks/data/dust_gu_pm10.csv')

In [9]:
# 구별 초미세먼지 : 나쁨 이상 횟수
dust4 = dust2[dust2['PM25'] > 35]
dust_gu = pd.DataFrame(dust4['gu'].value_counts())
dust_gu = dust_gu.reset_index()
dust_gu.rename(columns=({'index':'자치구', 'gu':'미세먼지나쁨일수'}), inplace=True)
dust_gu.to_csv('/content/drive/MyDrive/Colab Notebooks/data/dust_gu_pm25.csv')

In [10]:
# 구별 미세먼지 or 초미세먼지 : 나쁨 이상 횟수
dust5 = dust2[(dust2['PM10'] > 80) | (dust2['PM25'] > 35)]
dust_gu = pd.DataFrame(dust5['gu'].value_counts())
dust_gu = dust_gu.reset_index()
dust_gu.rename(columns=({'index':'자치구', 'gu':'미세먼지나쁨일수'}), inplace=True)
dust_gu.to_csv('/content/drive/MyDrive/Colab Notebooks/data/dust_gu_pm10_pm25.csv')

In [ ]:
dust_gu

### 구별/월별 데이터 처리

In [ ]:
# 구별 월별 미세먼지  : 나쁨 횟수
dust3['월'] = pd.to_datetime(dust3['date'].astype(str), format='%Y%m%d').dt.month
dust3_gp = dust3.groupby(['gu','월'])['date'].count()
dust3_gp = dust3_gp.reset_index()
dust3_gp.rename(columns={'gu':'자치구','date':'미세먼지나쁨일수'}, inplace=True)
dust3_gp.to_csv('/content/drive/MyDrive/Colab Notebooks/data/dust_gu_m_pm10.csv')
dust3_gp

In [ ]:
# 구별 월별 초미세먼지 : 나쁨 횟수
dust4['월'] = pd.to_datetime(dust4['date'].astype(str), format='%Y%m%d').dt.month
dust4_gp = dust4.groupby(['gu','월'])['date'].count()
dust4_gp = dust4_gp.reset_index()
dust4_gp.rename(columns={'date':'미세먼지나쁨일수'}, inplace=True)
dust4_gp.to_csv('/content/drive/MyDrive/Colab Notebooks/data/dust_gu_m_pm25.csv')
dust4_gp

In [ ]:
# 구별/월별 미세먼지 or 초미세먼지 : 나쁨 이상 횟수
dust5['월'] = pd.to_datetime(dust5['date'].astype(str), format='%Y%m%d').dt.month
dust5_gp = dust5.groupby(['gu','월'])['date'].count()
dust5_gp = dust5_gp.reset_index()
dust5_gp.rename(columns={'date':'미세먼지나쁨일수'}, inplace=True)
dust5_gp.to_csv('/content/drive/MyDrive/Colab Notebooks/data/dust_gu_m_pm10_pm25.csv')
dust5_gp

## 데이터 분석

### 상관 관계 분석

In [ ]:
dust = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_dust1620.csv')
dust.rename(columns={'gu': '자치구', 'count':'미세먼지나쁨일수'},inplace=True)
dust

In [ ]:
plt.scatter(dust['공원율'], dust['미세먼지나쁨일수'])
plt.xticks(rotation=90)
plt.show()

In [ ]:
dust.drop('연도', inplace=True, axis=1)
dust.head()

In [ ]:
_, axs = plt.subplots(2, 2, figsize=(18,8))
for i, feature in enumerate(dust.columns[2:]):
    row, col = i //2, i % 2
    sns.regplot(x=feature, y='미세먼지나쁨일수', data=dust, ci=None, ax=axs[row][col])

### 2019년 미세먼지 상관 관계

In [ ]:
dust19 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_dust2019.csv')
dust19.rename(columns={'gu': '자치구', 'date':'미세먼지나쁨일수'},inplace=True)
dust19

In [ ]:
_, axs = plt.subplots(2, 2, figsize=(18,8))
for i, feature in enumerate(dust19.columns[2:]):
    row, col = i //2, i % 2
    sns.regplot(x=feature, y='미세먼지나쁨일수', data=dust19, ci=None, ax=axs[row][col])

### LinearRegression

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_dust1620.csv')
df.drop('연도', axis=1, inplace=True)
df.head()

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
lr = LinearRegression()

In [ ]:
for feature in df.columns[2: ]:
    X = df[feature].values.reshape((-1,1))
    lr.fit(X, df['미세먼지나쁨일수'])
    score = lr.score(X,df['미세먼지나쁨일수'])
    print(f'{feature} : {score:.4f} \t{lr.coef_} \t{lr.intercept_}')

### MinMaxScaler

In [ ]:
df19 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_dust2019.csv')
df19.head()

In [ ]:
df19_a = np.array(df19.iloc[:,2:])
df19_a

In [ ]:
from sklearn.preprocessing import MinMaxScaler
dust19_a = np.array(df19.iloc[:,2:])
a19_mm = MinMaxScaler().fit_transform(df19_a)
df19_mm = pd.DataFrame(a19_mm, columns=df19.columns[2:])
df19_mm.describe()

In [ ]:
for feature in df19_mm.columns:
    X = df19_mm[feature].values.reshape((-1,1))
    lr.fit(X, df19['미세먼지나쁨일수'])
    score = lr.score(X,df19['미세먼지나쁨일수'])
    print(f'{feature} : {score:.4f} \t{lr.coef_} \t{lr.intercept_}')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_dust1619.csv')
df.head()

In [ ]:
for feature in df.columns[2: ]:
    X = df[feature].values.reshape((-1,1))
    lr.fit(X, df['미세먼지나쁨일수'])
    score = lr.score(X,df['미세먼지나쁨일수'])
    print(f'{feature} : {score:.4f} \t{lr.coef_} \t{lr.intercept_}')

### KMeans, silhouette_score

In [31]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [32]:
data = df19_mm.loc[:,['도로율','공원율']]
target = df19.loc[:,['자치구']]

In [ ]:
data.info()

In [34]:
sils = []
ints = []

for i in range(2, 10+1):
    kms = KMeans(n_clusters=i)
    kms.fit(data)
    ss = silhouette_score(data, kms.labels_, metric='euclidean')
    el = kms.inertia_

    sils.append(ss)
    ints.append(el)

In [ ]:
# 실루엣 계수 시각화
plt.plot(range(2, 10+1), sils, 'ro--')
plt.show()

In [ ]:
# 엘보우 계수 시각화
plt.plot(range(2, 10+1), ints, 'bo--')
plt.show()

In [ ]:
kms = KMeans(n_clusters=5, random_state=2209081630)
kms.fit(data)

In [ ]:
kms.labels_

In [ ]:
silhouette_score(data, kms.labels_, metric='euclidean')

In [ ]:
kms.inertia_

In [ ]:
centers = pd.DataFrame(kms.cluster_centers_, columns=['rd','pk'])
x, y = centers.rd, centers.pk

plt.scatter(np.array(data.iloc[:,0]), np.array(data.iloc[:,1]), c=kms.labels_)
plt.scatter(x, y, s=75,  marker='X', c='r') # 중심점 표시
plt.show()

In [ ]:
import plotly.express as px
df19 = df19.sort_values('미세먼지나쁨일수', ascending=False)

fig = px.scatter(df19, x="도로율", y="공원율", color="자치구",
                 size='미세먼지나쁨일수', height=700, width=800)

fig.show()

In [43]:
data = df19.loc[:,['공원율']]
target = df19.loc[:,['자치구']]

In [44]:
sils = []
ints = []

for i in range(2, 10+1):
    kms = KMeans(n_clusters=i)
    kms.fit(data)
    ss = silhouette_score(data, kms.labels_, metric='euclidean')
    el = kms.inertia_

    sils.append(ss)
    ints.append(el)

In [ ]:
# 실루엣 계수 시각화
plt.plot(range(2, 10+1), sils, 'ro--')
plt.show()

In [ ]:
# 엘보우 계수 시각화
plt.plot(range(2, 10+1), ints, 'bo--')
plt.show()

In [ ]:
kms = KMeans(n_clusters=6, random_state=2209131530)
kms.fit(data)

In [ ]:
kms.labels_

In [ ]:
silhouette_score(data, kms.labels_, metric='euclidean')

In [ ]:
kms.inertia_

## 데이터 시각화

### 구별 미세먼지 시각화

In [51]:
# 데이터 적재
dust_gu = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/dust_gu_pm10.csv')
seoul_geo = '/content/drive/MyDrive/Colab Notebooks/data/seoul_gu.json'

In [ ]:
seoul = [37.60, 126.97]
fmap = folium.Map(seoul, zoom_start=11)

folium.Choropleth(geo_data=seoul_geo, data=dust_gu, columns=['자치구', '미세먼지나쁨일수'], key_on='feature.properties.SGG_NM',
                  fill_color='YlOrBr', fill_opacity=0.6, legend_name='서울시 구별 미세먼지 정도').add_to(fmap)
fmap.save('/content/drive/MyDrive/Colab Notebooks/map/fmap01a.html')
fmap

### 구별 초미세먼지 시각화

In [53]:
# 데이터 적재
dust_gu = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/dust_gu_pm25.csv')
seoul_geo = '/content/drive/MyDrive/Colab Notebooks/data/seoul_gu.json'

In [ ]:
seoul = [37.60, 126.97]
fmap = folium.Map(seoul, zoom_start=11)

folium.Choropleth(geo_data=seoul_geo, data=dust_gu, columns=['자치구', '미세먼지나쁨일수'], key_on='feature.properties.SGG_NM',
                  fill_color='YlOrRd', fill_opacity=0.7, legend_name='서울시 구별 초미세먼지 정도').add_to(fmap)
fmap.save('/content/drive/MyDrive/Colab Notebooks/map/fmap01b.html')
fmap

### 구별/월별 미세먼지/초미세먼지 시각화

In [57]:
# 데이터 적재
dust_gu_m = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/dust_gu_m_pm10_pm25.csv')
seoul_geo = '/content/drive/MyDrive/Colab Notebooks/data/seoul_gu.json'

In [58]:
dust_gu_m.rename(columns={'gu':'자치구'}, inplace=True)

In [ ]:
gu_xy = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_gu.txt')
gus = dust_gu_m.자치구.unique()
print(gus)
gu_xy = gu_xy.sort_values(by='gu').reset_index().drop('index', axis=1)
gu_xy.rename(columns={'gu':'자치구'}, inplace=True)
gu_xy

In [ ]:
import folium
import altair as alt

seoul = [37.60, 126.97]

fmap = folium.Map(seoul, zoom_start=11)

folium.Choropleth(geo_data=seoul_geo, data=dust_gu, columns=['자치구', '미세먼지나쁨일수'], key_on='feature.properties.SGG_NM',
                   fill_color='YlOrRd', fill_opacity=0.6, legend_name='서울시 구별 미세먼지 정도').add_to(fmap)
fmap.save('/content/drive/MyDrive/Colab Notebooks/map/fmap01c.html')

# create an altair chart, then convert to JSON

for gu in gus:
    source = dust_gu_m[dust_gu_m['자치구'] == gu].loc[:,['월','미세먼지나쁨일수']].reset_index().drop('index',axis=1)
    chart = alt.Chart(source).mark_bar().encode(x='월', y='미세먼지나쁨일수')
    vis1 = chart.to_json()
    x = gu_xy[gu_xy['자치구'] == gu]['lat']
    y = gu_xy[gu_xy['자치구'] == gu]['lng']
    start_coords = [x, y]

    folium.Marker(
        location=start_coords,icon=folium.Icon(color="blue", icon="cloud"),
        popup=folium.Popup(max_width=450).add_child(
            folium.VegaLite(vis1, width=300, height=250)
        ),
    ).add_to(fmap)

fmap

### 공원율 시각화

In [ ]:
seoul_geo = '/content/drive/MyDrive/Colab Notebooks/data/seoul_gu.json'
park = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_park.csv')
park

In [ ]:
seoul = [37.56, 126.97]
fmap = folium.Map(seoul, zoom_start=11)

folium.Choropleth(geo_data=seoul_geo, data=park, columns=['gu', '공원율'], key_on='feature.properties.SGG_NM',
                  fill_color='YlGn', fill_opacity=0.6, legend_name='서울시 구별 공원율').add_to(fmap)
fmap.save('/content/drive/MyDrive/Colab Notebooks/map/fmap01.html')
fmap

### 도로율 시각화

In [ ]:
seoul_geo = '/content/drive/MyDrive/Colab Notebooks/data/seoul_gu.json'
road = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_road.csv')
road

In [ ]:
seoul = [37.60, 126.97]
fmap = folium.Map(seoul, zoom_start=11)

folium.Choropleth(geo_data=seoul_geo, data=road, columns=['gu', 'rate'], key_on='feature.properties.SGG_NM',
                  fill_color='Greys', fill_opacity=0.6, legend_name='서울시 구별 도로율').add_to(fmap)
fmap.save('/content/drive/MyDrive/Colab Notebooks/map/fmap02.html')
fmap

### 가로수량 시각화

In [ ]:
seoul_geo = open('/content/drive/MyDrive/Colab Notebooks/data/seoul_gu.json').read()
trees = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_trees.csv')
trees.columns

In [68]:
import json
geo_data = json.loads(seoul_geo)

In [ ]:
seoul = [37.60, 126.97]
fmap = folium.Map(seoul, zoom_start=11,show=False)

folium.Choropleth(geo_data=seoul_geo, data=trees, columns=['gu', '가로수량'], key_on='feature.properties.SGG_NM',
                  fill_color='YlGn', fill_opacity=0.6, legend_name='서울시 구별 가로수').add_to(fmap)
fmap.save('/content/drive/MyDrive/Colab Notebooks/map/fmap03.html')
fmap

### 서울시 구별 연도별 미세먼지 선 그래프 (2016 - 2020)

In [ ]:
dustyr = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/dust_year.csv')
dustyr.rename(columns={'gu':'자치구', 'date':'미세먼지나쁨일수', 'year': '연도'}, inplace=True)
dustyr.head()

In [ ]:
import plotly.express as px

fig = px.line(dustyr, x='자치구', y='미세먼지나쁨일수', color='연도', symbol="연도")
fig.show()

### 공원 면적 변화 시각화 (2016 - 2020)

In [ ]:
park_t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_park1620.csv')
park_t.head()

In [ ]:
import plotly.express as px

fig = px.line(park_t, x='자치구', y='공원율', color='연도', symbol="연도")
fig.show()

### 연도별 도로율 면적 변화 시각화 (2016 - 2020)

In [ ]:
road_t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_road1620.csv')
road_t.head()

In [ ]:
import plotly.express as px

fig = px.line(road_t, x='자치구', y='도로율', color='연도', symbol="연도")
fig.show()

### 연도별 가로수량

In [ ]:
tree_t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_tree1620.csv')
tree_t.head()

In [ ]:
import plotly.express as px

fig = px.line(tree_t, x='자치구', y='가로수량', color='연도', symbol="연도")
fig.show()

### 교통량 그래프(연도별)

In [ ]:
traffic_t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_traffic1620.csv')
traffic_t = traffic_t.sort_values('자치구')
traffic_t.head()

In [ ]:
import plotly.express as px

fig = px.line(traffic_t, x='자치구', y='교통량', color='연도', symbol="연도")
fig.show()

### 클러스터링 시각화

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/seoul_dust1619.csv')
df

In [91]:
pm = df.groupby(['자치구'])['미세먼지나쁨일수'].sum()
pr = df.groupby(['자치구'])['공원율'].mean().round(6)
guu = df['자치구'].unique()

In [ ]:
seoul_dust = pd.DataFrame({'자치구':guu, '미세먼지나쁨일수':pm, '공원율':pr})
seoul_dust

In [ ]:
color_bin = [233, 257, 290, 313, 336, 348]
seoul_dust['색상'] = pd.cut(seoul_dust['미세먼지나쁨일수'],bins=color_bin, labels=['lightgrey','yellow','green','red','black'])
seoul_dust

In [ ]:
seoul_dust['섹터'] = ['클러스터1' if s == '강서구' or s == '광진구' or s == '마포구' or s == '양천구'
                    else '클러스터2' if s == '강북구' or s == '도봉구' or s == '은평구' or s == '종로구'
                    else '클러스터3' if s == '관악구' or s == '노원구' or s == '서초구' or s == '성북구'
                                    or s == '중랑구'
                    else '클러스터4' if s == '강남구' or s == '강동구' or s == '구로구' or s == '금천구'
                                    or s == '동대문구' or s == '성동구' or s == '송파구' or s == '영등포구'
                                    or s == '용산구'
                    else '클러스터5' for s in seoul_dust['자치구']]
seoul_dust

In [ ]:
import plotly.express as px
fig = px.treemap(seoul_dust, path=[px.Constant("all"), '섹터', '자치구'],
                 values = '공원율', hover_data = ['미세먼지나쁨일수'])
fig.show()